## Amazon API Gateway, AWS Lambda and Amazon DynamoDB

This example deploys an Amazon API Gateway, AWS Lambda functions and an Amazon DynamoDB table using the AWS CDK and TypeScript. The AWS Lambda functions are deployed behind an Amazon API Gateway and execute for CRUD operations against Amazon DynamoDB. The Amazon DynamoDB table is partitioned on an accountid attribute and also includes a sort key on the vendorid attribute, together they form the primary key.

![architecture](../images/architecture_1.png "Architecture")

**Jupyter Notebook Scripts**
1.	The *Post Order* script create orders and posts them to the Amazon API Gateway

2.	The *json formatter* script creates a json formatter object which is used to render the json results.

3.	The *Get Order* script passes query string parameters to a Lambda function using the API Gateway. The function returns items that match the parameters

4.	The *Update Order* script, updates multiple items in the table.

5.	The *Get Updated Order* script is used again to validate the update operation.

6.	The Delete Order script is used to delete items from the DynamoDB table.

7.	The final Get Order script is used to validate the delete operation.


**Note:** Make sure you set **gwid** to your gateway id using - *gwid = '...'*

**Post Order**

In [ ]:
import boto3, requests, datetime

#Set gateway id
gwid = 'smz7s32gkj'

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/putorder')

for i in range(5):
    x = datetime.datetime.now()
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(x)
    coffeetype = 'Short Black'
    coffeesize = 'Small'
    unitprice = str(4.50 * (i+1))
    quantity = str(i+1)

    response = requests.post(url,json={'order':{
                'accountid': accountid,
                'vendorid': vendorid,
                'orderdate':orderdate,
                'details':{
                    'coffeetype': coffeetype,
                    'coffeesize': coffeesize,
                    'unitprice': unitprice,
                    'quantity': quantity
                }
            }
        })
    print(str(accountid) + ':' + str(vendorid))

**json formatter** - The following script create a class display nicely formatted json data

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

**Get Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**Update Order** in Amazon DynamoDB

In [ ]:
update_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/updateorder')


for i in range(2):
    x = datetime.datetime.now()
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(x)
    coffeetype = 'Flat white'
    coffeesize = 'Medium'
    unitprice = str(4.50 * (i+2))
    quantity = str(i+2)

    response = requests.post(update_url,json={'order':{
                'accountid': accountid,
                'vendorid': vendorid,
                'orderdate':orderdate,
                'details':{
                    'coffeetype': coffeetype,
                    'coffeesize': coffeesize,
                    'unitprice': unitprice,
                    'quantity': quantity
                }
            }
        })
    print(str(accountid) + ':' + str(vendorid))

**Get Updated Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**Delete Order** the order form Amazon DynamoDB

In [ ]:
delete_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/deleteorder')

response = requests.post(delete_url,json={'order':{'accountid':'a0','vendorid':'v0'}})

RenderJSON(response_get.json())

**Check Deleted** items in Amazon DynamoDB

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())